# Importing modules and libraries

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from bnn_model import *
from gm_model import *
import utils as ut
import edward as ed
from scipy.stats import norm
import tensorflow as tf

import os, sys

/home/arnaud/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Basic data loading

In [2]:
df_12H = pd.read_csv("./env/data_frames/final_data_frame_12H.csv", sep=";", index_col="timestamp", parse_dates=True)
df_30T = pd.read_csv("./env/data_frames/final_data_frame_30T.csv", sep=";", index_col="timestamp", parse_dates=True)
df_1D = pd.read_csv("./env/data_frames/final_data_frame_1D.csv", sep=";", index_col="timestamp", parse_dates=True)

In [3]:
df_12H.head()

,P1_361,P2_361,P1_92,P2_92,P1_146,P2_146,P1_205,P2_205,P1_325,P2_325,...,P1_270,P2_270,P1_217,P2_217,P1_SAKP,P2_SAKP,P1_SNTR,P2_SNTR,P1_SBC,P2_SBC
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,22.111298,10.243440,26.114637,12.047463,674.549166,217.105605,391.235397,163.518808,36.780570,28.645083,...,423.846957,282.256455,733.275467,372.112978,18.194899,10.817219,203.770042,190.713875,139.015875,135.773875
2017-01-01 12:00:00,22.111298,10.243440,26.114637,12.047463,440.326486,123.227917,276.615689,108.508503,30.299905,23.430454,...,171.686273,113.231669,442.801571,219.058729,18.194899,10.817219,142.579583,132.769042,115.455652,110.838833
2017-01-02 00:00:00,22.111298,10.243440,26.114637,12.047463,170.956361,40.115319,188.135729,60.922660,45.787858,31.553449,...,93.143708,53.855993,160.493576,69.264528,18.194899,10.817219,42.246625,24.358208,24.356750,23.402625
2017-01-02 12:00:00,19.868429,9.346914,26.114637,12.047463,33.017775,13.010987,56.738035,24.496993,20.961917,14.012965,...,24.302650,18.269826,37.356400,19.790139,18.194899,10.817219,94.536375,26.833708,24.753125,21.749625
2017-01-03 00:00:00,30.423688,12.067064,26.114637,12.047463,44.160487,15.140202,45.049146,20.395583,24.165230,19.546546,...,23.022368,16.531896,47.165326,23.431062,18.194899,10.817219,13.051375,3.614792,3.533833,2.509417


In [4]:
df_30T.head()

,P1_462,P2_462,P1_438,P2_438,P1_361,P2_361,P1_92,P2_92,P1_146,P2_146,...,P1_217,P2_217,P1_466,P2_466,P1_SAKP,P2_SAKP,P1_SNTR,P2_SNTR,P1_SBC,P2_SBC
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,16.366499,13.482981,12.938215,8.846571,22.11639,10.247138,26.015849,12.011004,836.805000,298.463793,...,966.601481,489.250556,21.421378,9.178647,16.880193,11.383044,58.131,56.141,74.171,72.986
2017-01-01 00:30:00,16.366499,13.482981,12.938215,8.846571,22.11639,10.247138,26.015849,12.011004,888.627500,314.431667,...,858.659000,443.253500,21.421378,9.178647,16.880193,11.383044,110.584,104.177,140.157,134.698
2017-01-01 01:00:00,16.366499,13.482981,12.938215,8.846571,22.11639,10.247138,26.015849,12.011004,869.985000,304.948000,...,889.605833,446.463667,21.421378,9.178647,16.880193,11.383044,223.940,203.503,183.083,177.123
2017-01-01 01:30:00,16.366499,13.482981,12.938215,8.846571,22.11639,10.247138,26.015849,12.011004,837.401167,287.329500,...,976.471500,486.042167,21.421378,9.178647,16.880193,11.383044,238.947,217.523,155.423,149.313
2017-01-01 02:00:00,16.366499,13.482981,12.938215,8.846571,22.11639,10.247138,26.015849,12.011004,781.867000,266.075167,...,889.134500,450.835500,21.421378,9.178647,16.880193,11.383044,250.207,228.720,148.580,137.223


In [5]:
df_1D.head()

,P1_462,P2_462,P1_529,P2_529,P1_438,P2_438,P1_361,P2_361,P1_92,P2_92,...,P1_217,P2_217,P1_466,P2_466,P1_SAKP,P2_SAKP,P1_SNTR,P2_SNTR,P1_SBC,P2_SBC
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01,16.325911,13.437303,7.729455,5.678755,12.90083,8.821739,22.079663,10.230709,26.397212,12.133722,...,588.139871,295.639257,21.548377,9.213323,17.9672,10.762696,173.174813,161.741458,127.486404,123.306354
2017-01-02,16.325911,13.437303,7.729455,5.678755,12.90083,8.821739,19.868429,9.346914,26.397212,12.133722,...,99.010619,44.561739,21.548377,9.213323,17.9672,10.762696,68.391500,25.595958,24.554938,22.576125
2017-01-03,16.325911,13.437303,7.729455,5.678755,12.90083,8.821739,25.245851,10.571003,26.397212,12.133722,...,39.636635,20.153208,21.548377,9.213323,17.9672,10.762696,12.884979,3.623354,3.866468,2.481660
2017-01-04,16.325911,13.437303,7.729455,5.678755,12.90083,8.821739,9.389965,5.320046,26.397212,12.133722,...,15.908250,8.693816,21.548377,9.213323,17.9672,10.762696,29.057854,13.207292,21.220979,13.132417
2017-01-05,16.325911,13.437303,7.729455,5.678755,12.90083,8.821739,9.562080,5.296205,16.318983,6.804903,...,20.632745,10.526028,21.548377,9.213323,17.9672,10.762696,14.964646,7.145833,7.521146,6.341438


In [6]:
P1_colmns_1D = list(filter(lambda col: "P1" in str(col),list(df_1D.columns.values)))
P2_colmns_1D = list(filter(lambda col: "P2" in str(col),list(df_1D.columns.values)))

P1_colmns_12H = list(filter(lambda col: "P1" in str(col),list(df_12H.columns.values)))
P2_colmns_12H = list(filter(lambda col: "P2" in str(col),list(df_12H.columns.values)))

P1_colmns_30T= list(filter(lambda col: "P1" in str(col),list(df_30T.columns.values)))
P2_colmns_30T = list(filter(lambda col: "P2" in str(col),list(df_30T.columns.values)))

np.array([P1_colmns_1D, P2_colmns_1D]).T

array([['P1_462', 'P2_462'],
       ['P1_529', 'P2_529'],
       ['P1_438', 'P2_438'],
       ['P1_361', 'P2_361'],
       ['P1_92', 'P2_92'],
       ['P1_215', 'P2_215'],
       ['P1_146', 'P2_146'],
       ['P1_205', 'P2_205'],
       ['P1_325', 'P2_325'],
       ['P1_309', 'P2_309'],
       ['P1_237', 'P2_237'],
       ['P1_261', 'P2_261'],
       ['P1_197', 'P2_197'],
       ['P1_426', 'P2_426'],
       ['P1_251', 'P2_251'],
       ['P1_249', 'P2_249'],
       ['P1_271', 'P2_271'],
       ['P1_422', 'P2_422'],
       ['P1_227', 'P2_227'],
       ['P1_434', 'P2_434'],
       ['P1_343', 'P2_343'],
       ['P1_209', 'P2_209'],
       ['P1_317', 'P2_317'],
       ['P1_351', 'P2_351'],
       ['P1_363', 'P2_363'],
       ['P1_359', 'P2_359'],
       ['P1_211', 'P2_211'],
       ['P1_219', 'P2_219'],
       ['P1_458', 'P2_458'],
       ['P1_413', 'P2_413'],
       ['P1_335', 'P2_335'],
       ['P1_428', 'P2_428'],
       ['P1_345', 'P2_345'],
       ['P1_185', 'P2_185'],
       ['P1_515'

In [7]:
X_1D = df_1D[P1_colmns_1D[0:-3]].values
y_SBC_1D = df_1D[P1_colmns_1D[-1]].values
y_SNTR_1D = df_1D[P1_colmns_1D[-2]].values

X_30T = df_30T[P1_colmns_30T[0:-3]].values
y_SBC_30T = df_30T[P1_colmns_30T[-1]].values
y_SNTR_30T = df_30T[P1_colmns_30T[-2]].values

X_12H = df_12H[P1_colmns_12H[0:-3]].values
y_SBC_12H = df_12H[P1_colmns_12H[-1]].values
y_SNTR_12H = df_12H[P1_colmns_12H[-2]].values

In [8]:
def test_train_split(X, y, train_size=0.75):
        train_cnt = int(round(X.shape[0]*0.75, 0))
        return X[0:train_cnt], X[train_cnt:], y[0:train_cnt], y[train_cnt:] 
        
        
        

In [9]:

train_per = 0.75
X_train, X_test, y_train, y_test = test_train_split(X_1D, y_SBC_1D,train_size=train_per)
#X_train, X_test, y_train, y_test = test_train_split(X_1D, y_NTTR_1D)

#X_train, X_test, y_train, y_test = test_train_split(X_1D, y_SBC_12H)
#X_train, X_test, y_train, y_test = test_train_split(X_1D, y_NTTR_12H)



print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(274, 73)
(91, 73)
(274,)
(91,)


In [10]:
station = "SBC"

In [11]:
neurons_per_inner_bnn = 5
inner_layer_cnt_bnn = 2


batch_size_bnn = 200
epochs_bnn = 1000
updates_per_batch_bnn = 5

layers = []
for i in range(updates_per_batch_bnn):
    layers.append(neurons_per_inner_bnn)

model_id = "bnn_n"+str(neurons_per_inner_bnn)+"_l"+str(inner_layer_cnt_bnn)+"_s"+str(station)+"_tts" + str(train_per)
model = Bnn(model_id)


print("Model id:" + model_id)




Model id:bnn_n5_l2_sSBC_tts0.75


In [ ]:
#model.load("./env/models/" + model_id +"/", model_id)
model.build(2,1, layers)

In [13]:
#model.fit(X_train, y_train, M=batch_size, updates_per_batch=updates_per_batch, epochs=epochs)


In [14]:
Some basic tests on the test set

In [15]:
#model.save("./env/models/" + model_id +"/", model_id)

In [16]:
neurons_per_inner_mdn = 5
inner_layer_cnt_mdn = 2
mixtures_cnt = 2

batch_size_mdn = 200
epochs_mdn = 1000
updates_per_batch_mdn = 5

layers_mdn = []
for i in range(inner_layer_cnt_mdn):
    layers.append(neurons_per_inner_mdn)

model_id = "mdn_n"+str(neurons_per_inner_mdn)+"_l"+str(inner_layer_cnt_mdn)+"_m"+str(mixtures_cnt)+"_s"+str(station)+"_tts" + str(train_per)

mdn_model = Mdn(model_id, X_train, y_train, inner_dims=layers_mdn, num_mixtures=mixtures_cnt)

print("Model id:" + model_id)



Model id:mdn_n5_l2_m2_sSBC_tts0.75


In [17]:
#mdb_model.fit(num_iter=10000)

In [18]:
#mdn_model.load("./env/models/" + model_id +"/", model_id)

In [19]:
#mdn_model.save("./env/models/" + model_id +"/", model_id)